In [3]:
#Imports

from collections import Counter

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
import time
import tqdm
import json
import re
import pandas as pd

In [4]:
# importing the module
import json
  
# Opening JSON file
f_train = open('json_datasets/train.json', 'r')
raw_train = json.load(f_train)
f_train.close()

f_test = open('json_datasets/test.json', 'r')
raw_test = json.load(f_test)
f_test.close()

In [5]:
def removeQuotes(lst):
    ret = []
    for s in lst:
        if s != '``' and s != "''":
            ret.append(s)
    return ret

def camelCaseSplit(identifier):
    matches = re.finditer('.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)', identifier)
    d = [m.group(0) for m in matches]
    new_d = []
    for token in d:
        token = token.replace('(', '')
        token = token.replace(')', '')
        token_split = token.split('_')
        for t in token_split:
            #new_d.append(t.lower())
            new_d.append(t)
    return new_d

In [6]:
def g2tPreprocess(raw):
    df = []
    for item in raw:
        graph = 'g2t:'
        for relation in item['relations']:
            graph += ' <H> ' + ' '.join(removeQuotes(relation[0])) + ' <R> '
            graph += ' '.join(camelCaseSplit(relation[1])) + ' <T> '
            graph += ' '.join(removeQuotes(relation[2]))

        ents = [' '.join(removeQuotes(entity)) for entity in item['entities']]
        text = item['text']
        for i in range(len(ents)):
            text = text.replace('<ENT_'+str(i)+'>', ents[i])
        sample = [graph, text]
        df.append(sample)
    return pd.DataFrame(df, columns=['source_text', 'target_text'])

In [7]:
train_df = g2tPreprocess(raw_train)
test_df = g2tPreprocess(raw_test)
print(test_df)

source_text  \
0     g2t: <H> Abilene Regional Airport <R> city Ser...   
1     g2t: <H> Abilene Regional Airport <R> city Ser...   
2     g2t: <H> Adolfo Suárez Madrid–Barajas Airport ...   
3     g2t: <H> Adolfo Suárez Madrid–Barajas Airport ...   
4     g2t: <H> Adolfo Suárez Madrid–Barajas Airport ...   
...                                                 ...   
4923  g2t: <H> Twilight ( band ) <R> genre <T> Black...   
4924  g2t: <H> Twilight ( band ) <R> genre <T> Black...   
4925  g2t: <H> Uruguay <R> leader Name <T> Raúl Fern...   
4926  g2t: <H> Uruguay <R> leader Name <T> Raúl Fern...   
4927  g2t: <H> Uruguay <R> leader Name <T> Raúl Fern...   

                                            target_text  
0     Abilene , Texas is served by the Abilene Regio...  
1     Abilene Regional Airport serves the city of Ab...  
2     Adolfo Suárez Madrid–Barajas Airport can be fo...  
3     Adolfo Suárez Madrid–Barajas Airport is locate...  
4     Adolfo Suárez Madrid–Barajas Airport is

In [12]:
pip install --upgrade simplet5

Requirement already up-to-date: simplet5 in /Users/ramisbahi/opt/miniconda3/lib/python3.8/site-packages (0.1.3)
Note: you may need to restart the kernel to use updated packages.


In [13]:
from simplet5 import SimpleT5

# instantiate
model = SimpleT5()

# load (supports t5, mt5, byT5 models)
model.from_pretrained("t5","t5-base")

In [20]:
model.load_model("t5","./simplet5", use_gpu=True)

404 Client Error: Not Found for url: https://huggingface.co/simplet5/resolve/main/config.json


OSError: Can't load config for './simplet5'. Make sure that:

- './simplet5' is a correct model identifier listed on 'https://huggingface.co/models'

- or './simplet5' is the correct path to a directory containing a config.json file



In [16]:
f_dev = open('json_datasets/dev.json', 'r')
raw_dev = json.load(f_dev)
f_dev.close()

In [17]:
dev_df = g2tPreprocess(raw_dev)

In [18]:
dev_df

,source_text,target_text
0,g2t: <H> Aarhus <R> leader Name <T> Jacob Bund...,the leader of Aarhus is Jacob Bundsgaard .
1,g2t: <H> Aarhus Airport <R> runway Length <T> ...,Aarhus Airport 's runway length is 2702.0 .
2,g2t: <H> Aarhus Airport <R> runway Length <T> ...,the Aarhus Airport has a runway length of 2702...
3,g2t: <H> Adirondack Regional Airport <R> eleva...,Adirondack Regional Airport is 507 metres abov...
4,g2t: <H> Adirondack Regional Airport <R> locat...,Adirondack Regional Airport is located at Harr...
...,...,...
1637,g2t: <H> Acharya Institute of Technology <R> p...,the Acharya Institute of Technology is in Bang...
1638,g2t: <H> Acharya Institute of Technology <R> p...,the Acharya Institute of Technology was establ...
1639,g2t: <H> Romania <R> ethnic Group <T> Germans ...,1 Decembrie 1918 University is located in the ...
1640,g2t: <H> Visvesvaraya Technological University...,in the year 2000 the Acharya Institute of Tech...


In [19]:
# from pycocoevalcap.bleu.bleu import Bleu
#bleu = Bleu(4)

dev_df = g2tPreprocess(raw_dev)
# print(dev_df)

#hyp = [model.predict(dev_df['source_text'][i])[0] for i in range(len(dev_df))]

hyp = model.predict(dev_df['source_text'][0])
# ret = bleu.compute_score(dev_df['target_text'], hyp)
print(dev_df['source_text'][0])
print(hyp)
# print(dev_df['source_text'][50])
# print(model.predict(dev_df['source_text'][50]))

AttributeError: 'SimpleT5' object has no attribute 'device'

In [14]:
print(dev_df['target_text'][0])

the leader of Aarhus is Jacob Bundsgaard .


In [22]:
dev_df['source_text'].tolist()

onal Airport <R> runway Length <T> 1533.0 <H> Alpena County Regional Airport <R> elevation Above The Sea Level in metres <T> 210',
 'g2t: <H> Alpena County Regional Airport <R> location <T> Wilson Township , Alpena County , Michigan <H> Alpena County Regional Airport <R> runway Length <T> 1533.0 <H> Alpena County Regional Airport <R> elevation Above The Sea Level in metres <T> 210',
 'g2t: <H> Amsterdam Airport Schiphol <R> city Served <T> Amsterdam <H> Amsterdam Airport Schiphol <R> runway Length <T> 3500.0 <H> Amsterdam Airport Schiphol <R> elevation Above The Sea Level in metres <T> -3.3528',
 'g2t: <H> Andrews County Airport <R> location <T> Texas <H> Texas <R> largest City <T> Houston <H> Texas <R> country <T> United States',
 'g2t: <H> Angola International Airport <R> location <T> Ícolo e Bengo <H> Ícolo e Bengo <R> country <T> Angola <H> Angola International Airport <R> runway Length <T> 4000.0',
 'g2t: <H> Angola International Airport <R> location <T> Ícolo e Bengo <H> Ícolo e 

In [15]:
pip install "git+https://github.com/salaniz/pycocoevalcap.git"

  Cloning https://github.com/salaniz/pycocoevalcap.git to /tmp/pip-req-build-0g2pptlc
  Running command git clone -q https://github.com/salaniz/pycocoevalcap.git /tmp/pip-req-build-0g2pptlc
  Created wheel for pycocoevalcap: filename=pycocoevalcap-1.2-py3-none-any.whl size=104312215 sha256=6a2455bbbf0c18d9078f4631054b9f29fd6c42179ed38ee89dabe77fb2ea5277
  Stored in directory: /tmp/pip-ephem-wheel-cache-8n19k4e1/wheels/6f/c9/51/e266f0496048c16686e133d8e33644d692931a356bfb372aae
Successfully built pycocoevalcap


In [1]:
from collections import defaultdict

ref = defaultdict(list)
ptr = 0
same = defaultdict(list)

for i in range(len(dev_df)):
    if i > 0 and dev_df['source_text'][i] != dev_df['source_text'][i-1]:
        ptr += 1
    same[ptr].append(dev_df['target_text'][i].lower())
    ref[i] = same[ptr]
    

NameError: name 'dev_df' is not defined

In [24]:
from pycocoevalcap.bleu.bleu import Bleu
from pycocoevalcap.rouge.rouge import Rouge
from pycocoevalcap.cider.cider import Cider
from pycocoevalcap.meteor.meteor import Meteor

bleu = Bleu(4)
meteor = Meteor()
rouge = Rouge()
cider = Cider()

hyp = [model.predict(dev_df['source_text'][i])[0] for i in range(len(dev_df))]

hyp = dict(zip(range(len(dev_df)), [[x.lower()] for x in hyp]))
# ref = dict(zip(range(len(dev_df)), [[dev_df['target_text'][i]] for i in range(len(dev_df))]))
ret = bleu.compute_score(ref, hyp)
print('BLEU INP {0:}'.format(len(hyp)))
print('BLEU 1-4 {0:}'.format(ret[0]))
print('METEOR {0:}'.format(meteor.compute_score(ref, hyp)[0]))
print('ROUGE_L {0:}'.format(rouge.compute_score(ref, hyp)[0]))
print('Cider {0:}'.format(cider.compute_score(ref, hyp)[0]))

{'testlen': 28884, 'reflen': 32048, 'guess': [28884, 27242, 25600, 23958], 'correct': [23149, 16868, 12291, 8993]}
ratio: 0.9012730903644252
BLEU INP 1642
BLEU 1-4 [0.7182929101500463, 0.6313590823666462, 0.5556170953663785, 0.49011938322845827]
METEOR 0.4636927254205504
ROUGE_L 0.6282289869288418
Cider 3.08811488391188
